## 0.準備

リポジトリをクローン

In [0]:
!git clone https://github.com/NON906/NVC_train.git

リポジトリのフォルダ内へ移動

In [0]:
%cd NVC_train

Googleドライブに接続

In [0]:
# 参考：https://qiita.com/tomo_makes/items/b3c60b10f7b25a0a5935

# google-drive-ocamlfuseのインストール
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Colab用のAuth token作成
from google.colab import auth
auth.authenticate_user()

# Drive FUSE library用のcredential生成
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# drive/ を作り、そこにGoogle Driveをマウントする
!mkdir -p drive
!google-drive-ocamlfuse drive

## 1. 音声から解析ファイルを生成

In [0]:
# soxのインストール
!apt-get -y install sox

# SPTKのインストール
!apt-get -y install csh
!wget http://downloads.sourceforge.net/sp-tk/SPTK-3.11.tar.gz
!tar xfvz SPTK-3.11.tar.gz
%cd SPTK-3.11
!./configure
!make
!make install
%cd ..

In [0]:
from google.colab import files
import os

# 音声の入ったzipファイルをアップロード
uploaded = files.upload()

for fn in uploaded.keys():
    os.rename(fn, 'targets.zip')

!unzip targets.zip -d targets

In [0]:
# nvmファイルの生成（nvzファイルに必要）
%run -i make_nvm.py targets drive/nvc/target.nvm

# voice.h5のダウンロード・解凍
!wget https://github.com/NON906/NVC_train/releases/download/v0.1/voice_h5.zip
!unzip voice_h5.zip

# gen_targetsの生成（学習に必要）
%run -i gen_targets.py targets gen_targets drive/nvc/gen_targets.zip

## 2. 学習データtarget.h5の生成（GPU推奨）

In [0]:
%run -i target_train.py gen_targets drive/nvc/target.h5 20 -1 32

（備考：タイムアウトなどで中断した場合は、以下で再開できる）

In [0]:
# gen_targetsを解凍・再配置
!unzip drive/nvc/gen_targets.zip -d gen_targets

%run -i target_retrain.py drive/nvc/target.h5 gen_targets drive/nvc/target.h5 20 -1 32

## 3. 学習データpitch.h5の生成（GPU推奨）

In [0]:
%run -i target_pitch_train.py gen_targets drive/nvc/pitch.h5 20 -1 32

（備考：タイムアウトなどで中断した場合は、以下で再開できる）

In [0]:
# gen_targetsを解凍・再配置
!unzip drive/nvc/gen_targets.zip -d gen_targets

%run -i target_pitch_retrain.py drive/nvc/pitch.h5 gen_targets drive/nvc/pitch.h5 20 -1 32

## 4. pbファイルを作成し、nvzファイルに統合

In [0]:
# 新しいバージョンだと、TensorFlowSharpでエラーが出るので、古いバージョンを用意する
# （注意：「!kill -9 -1」でリセットするまで、他の処理が行えなくなります）
!pip install tensorflow==1.5.0

In [0]:
%run -i target_convert_h5_to_pb.py drive/nvc/target.h5 drive/nvc/target.pb
%run -i target_pitch_convert_h5_to_pb.py drive/nvc/pitch.h5 drive/nvc/pitch.pb

%run -i make_nvz.py drive/nvc/target.nvz drive/nvc/target.nvm drive/nvc/target.pb drive/nvc/pitch.pb